In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date, lit, date_format

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Installments") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 17:59:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

installments = spark.read.csv("/data/raw/installments_payments.csv",
                               header=True,
                               inferSchema=True)

installments.createOrReplaceTempView("installments")

# Contagem de linhas e colunas
num_rows = installments.count()
num_columns = len(installments.columns)

print(f'Quantidade de linhas: {num_rows}')
print(f'Quantidade de variaveis (colunas): {num_columns}')

installments.show(5, truncate=False)

Quantidade de linhas: 13605401
Quantidade de variaveis (colunas): 8
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|1054186   |161674    |1.0                   |6                    |-1180.0        |-1187.0           |6948.36       |6948.36    |
|1330831   |151639    |0.0                   |34                   |-2156.0        |-2156.0           |1716.525      |1716.525   |
|2085231   |193053    |2.0                   |1                    |-63.0          |-63.0             |25425.0       |25425.0    |
|2452527   |199697    |1.0                   |3                    |-2418.0        |-2426.0           |24350.13      |24350.13   |
|2714724   |167

## Criando flags de janela temporal

In [3]:
df_temp_01 = spark.sql('''
  SELECT
    *,
    CASE WHEN DAYS_INSTALMENT >= -90 THEN 1 ELSE 0 END AS U3M,
    CASE WHEN DAYS_INSTALMENT >= -180 THEN 1 ELSE 0 END AS U6M,    
    CASE WHEN DAYS_INSTALMENT >= -360 THEN 1 ELSE 0 END AS U12M
  FROM installments
  ORDER BY SK_ID_PREV
''')

df_temp_01.createOrReplaceTempView('df_temp_01')
display(df_temp_01.limit(5))

DataFrame[SK_ID_PREV: int, SK_ID_CURR: int, NUM_INSTALMENT_VERSION: double, NUM_INSTALMENT_NUMBER: int, DAYS_INSTALMENT: double, DAYS_ENTRY_PAYMENT: double, AMT_INSTALMENT: double, AMT_PAYMENT: double, U3M: int, U6M: int, U12M: int]

## Criando variáveis de primeira camada

In [4]:
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('NUM_INSTALMENT_VERSION')
colunas_agregacao_total.remove('NUM_INSTALMENT_NUMBER')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'DAYS' in coluna:
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_INSTALLMENTS'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_INSTALLMENTS'))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_INSTALLMENTS'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_INSTALLMENTS'))  


# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
display(df_temp_02.limit(5))



Quantidade Total de Variáveis Criadas: 24
Nomes das Variáveis Criadas: ['QT_MAX_DAYS_INSTALMENT_U3M_INSTALLMENTS', 'QT_MIN_DAYS_INSTALMENT_U3M_INSTALLMENTS', 'QT_MAX_DAYS_INSTALMENT_U6M_INSTALLMENTS', 'QT_MIN_DAYS_INSTALMENT_U6M_INSTALLMENTS', 'QT_MAX_DAYS_INSTALMENT_U12M_INSTALLMENTS', 'QT_MIN_DAYS_INSTALMENT_U12M_INSTALLMENTS', 'QT_MAX_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS', 'QT_MIN_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS', 'QT_MAX_DAYS_ENTRY_PAYMENT_U6M_INSTALLMENTS', 'QT_MIN_DAYS_ENTRY_PAYMENT_U6M_INSTALLMENTS', 'QT_MAX_DAYS_ENTRY_PAYMENT_U12M_INSTALLMENTS', 'QT_MIN_DAYS_ENTRY_PAYMENT_U12M_INSTALLMENTS', 'VL_TOT_AMT_INSTALMENT_U3M_INSTALLMENTS', 'VL_MED_AMT_INSTALMENT_U3M_INSTALLMENTS', 'VL_TOT_AMT_INSTALMENT_U6M_INSTALLMENTS', 'VL_MED_AMT_INSTALMENT_U6M_INSTALLMENTS', 'VL_TOT_AMT_INSTALMENT_U12M_INSTALLMENTS', 'VL_MED_AMT_INSTALMENT_U12M_INSTALLMENTS', 'VL_TOT_AMT_PAYMENT_U3M_INSTALLMENTS', 'VL_MED_AMT_PAYMENT_U3M_INSTALLMENTS', 'VL_TOT_AMT_PAYMENT_U6M_INSTALLMENTS', 'VL_MED_AMT_PAYMENT

[Stage 8:==============>                                            (2 + 6) / 8]

Quantidade de linhas do DataFrame: 997752
Quantidade de colunas do DataFrame: 25




DataFrame[SK_ID_PREV: int, QT_MAX_DAYS_INSTALMENT_U3M_INSTALLMENTS: double, QT_MIN_DAYS_INSTALMENT_U3M_INSTALLMENTS: double, QT_MAX_DAYS_INSTALMENT_U6M_INSTALLMENTS: double, QT_MIN_DAYS_INSTALMENT_U6M_INSTALLMENTS: double, QT_MAX_DAYS_INSTALMENT_U12M_INSTALLMENTS: double, QT_MIN_DAYS_INSTALMENT_U12M_INSTALLMENTS: double, QT_MAX_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS: double, QT_MIN_DAYS_ENTRY_PAYMENT_U3M_INSTALLMENTS: double, QT_MAX_DAYS_ENTRY_PAYMENT_U6M_INSTALLMENTS: double, QT_MIN_DAYS_ENTRY_PAYMENT_U6M_INSTALLMENTS: double, QT_MAX_DAYS_ENTRY_PAYMENT_U12M_INSTALLMENTS: double, QT_MIN_DAYS_ENTRY_PAYMENT_U12M_INSTALLMENTS: double, VL_TOT_AMT_INSTALMENT_U3M_INSTALLMENTS: double, VL_MED_AMT_INSTALMENT_U3M_INSTALLMENTS: double, VL_TOT_AMT_INSTALMENT_U6M_INSTALLMENTS: double, VL_MED_AMT_INSTALMENT_U6M_INSTALLMENTS: double, VL_TOT_AMT_INSTALMENT_U12M_INSTALLMENTS: double, VL_MED_AMT_INSTALMENT_U12M_INSTALLMENTS: double, VL_TOT_AMT_PAYMENT_U3M_INSTALLMENTS: double, VL_MED_AMT_PAYMENT_U3M_INSTA

In [5]:
df_temp_02.write.mode("overwrite").parquet('/data/books/installments')


25/07/21 18:00:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [6]:
spark.stop()